In [61]:
import ROOT as r
import numpy as np 
def Unpack_Histo(histogram_file):
    bin_content = []
    bin_content_uncer =[]
    for i in range(1,histogram_file.GetNbinsX()+1):
        bin_content.append(histogram_file.GetBinContent(i))
        bin_content_uncer.append(histogram_file.GetBinError(i))
        
    return np.array(bin_content),np.array(bin_content_uncer)

############# CONFIGURING THE JOB ###########################
samples_path = "/home/diego/HEP/VBF/MuMu/Plots/V26-SR/"
sample1_name = "Zmumu_SherpaSAMERW"
sample2_name = "Zmumu_MGRW"



In [62]:
r.TH1.AddDirectory(r.kFALSE)
file1 = r.TFile.Open(samples_path+sample1_name+'.root')
file2 = r.TFile.Open(samples_path+sample2_name+'.root')

myFile =r.TFile.Open("Average.root", "RECREATE")

histos1 = []
histos2 = []

for i in file1.GetListOfKeys():
    histos1.append(str(i.GetName()))
    
for i in file2.GetListOfKeys():
    histos2.append(str(i.GetName()))
    
    
for histo in histos1 :
    if histo in histos2:
        
        bin_range = [(file1.Get(histo)).GetXaxis().GetXmin(),file1.Get(histo).GetXaxis().GetXmax()]
        
        content1, uncer1 = Unpack_Histo(file1.Get(histo))
        content2, uncer2 = Unpack_Histo(file2.Get(histo))
        
        new_content = []
        new_uncer = []
        
        for i in range(0,len(content1)):
            if content1[i]!=0 and content2[i]!=0:
                new_content.append((content1[i]/uncer1[i]**2+content2[i]/uncer2[i]**2)/(1/uncer1[i]**2+1/uncer2[i]**2))
                new_uncer.append(1/(1/uncer1[i]**2+1/uncer2[i]**2))
            elif content1[i]!=0:
                new_content.append(content2[i])
                new_uncer.append(uncer2[i])
            elif content2[i]!=0:
                new_content.append(content1[i])
                new_uncer.append(uncer1[i])
            else :
                new_content.append(0.0)
                new_uncer.append(0.0)
                
        r.TH1.AddDirectory(r.kFALSE)
        final_hist = r.TH1F(histo,histo,len(new_content),bin_range[0],bin_range[1])
        
        for i in range(1,final_hist.GetNbinsX()+1):
            
            final_hist.SetBinContent(i,new_content[i-1])
            final_hist.SetBinError(i,new_uncer[i-1])
            
        myFile.WriteObject(final_hist,histo)
     
myFile.Close()

In [23]:
histos1

['lep1_pt_basic',
 'lep1_pt_basic_dphi',
 'lep1_pt_basic_dphi_drap',
 'lep1_pt_basic_dphi_drap_btag',
 'lep1_pt_basic_dphi_drap_btag_iso',
 'lep1_pt_basic_dphi_drap_btag_iso_pt1',
 'lep1_pt_basic_dphi_drap_btag_iso_pt1_pt2',
 'lep1_pt_basic_dphi_drap_btag_iso_pt1_pt2_j1pt',
 'lep1_pt_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt',
 'lep1_pt_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal',
 'lep1_pt_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj',
 'lep1_pt_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji',
 'lep1_pt_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen',
 'lep1_pt_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass',
 'lep1_pt_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl',
 'lep2_pt_basic',
 'lep2_pt_basic_dphi',
 'lep2_pt_basic_dphi_drap',
 'lep2_pt_basic_dphi_drap_btag',
 'lep2_pt_basic_dphi_drap_btag_iso',
 'lep2_pt_basic_dphi_drap_btag_iso_pt1',
 'lep2_pt_basic_dphi_drap_btag_iso_pt1_pt2',
 'lep2_pt_basi